In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
# from tensorflow import

## 【問題1】スクラッチを振り返る

- 重みの初期化
- エポックのループ
- optimizerの指定

## 【問題2】スクラッチとTensorFlowの対応を考える

- 重みの初期化: tf.Variableで宣言して、tf.global_variable_initializerメソッドで初期化
- エポックのループ: for loop
- optimizerの指定: AdamOptimizerクラスを使用

## 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成

In [2]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [3]:
def example_net(x):
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
    return layer_output

In [4]:
data = load_iris()
X = pd.DataFrame(data.data, columns=data.feature_names, dtype=np.float32).values
y = pd.get_dummies(data.target, dtype=np.float32).values

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [6]:
learning_rate = 0.01
n_input = X_train.shape[1]
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_classes = 3

X = tf.placeholder('float', [None, n_input])
Y = tf.placeholder('float', [None, n_classes])

logits = example_net(X)
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

correct_pred = tf.equal(tf.argmax(Y), tf.argmax(tf.nn.softmax(logits)))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

kf = KFold(n_splits=4, shuffle=True)
for train_idx, valid_idx in kf.split(X_train, y_train):
    X_tr, X_val = X_train[train_idx], X_train[valid_idx]
    y_tr, y_val = y_train[train_idx], y_train[valid_idx]
    n_samples, n_input = X_tr.shape
    get_mini_batch_train = GetMiniBatch(X_tr, y_tr, batch_size=batch_size)
    
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(num_epochs):
            total_batch  = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
            total_loss = 0
            total_acc = 0
            for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
                sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
                loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
                total_loss += loss
                total_acc += acc
            total_loss /= n_samples
            total_acc /= n_samples
            val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
            print(f"Epoch {epoch}, loss : {loss:.4f}, val_loss : {val_loss:.4f}, acc : {acc:.3f}, val_acc : {val_acc:.3f}")
        test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
        print(f'test_acc: {test_acc:.3f}')

Epoch 0, loss : 10.5060, val_loss : 25.4051, acc : 0.333, val_acc : 0.000
Epoch 1, loss : 8.1467, val_loss : 10.1863, acc : 0.333, val_acc : 0.667
Epoch 2, loss : 3.1350, val_loss : 3.1067, acc : 0.333, val_acc : 0.333
Epoch 3, loss : 3.7615, val_loss : 8.6068, acc : 0.667, val_acc : 0.333
Epoch 4, loss : 1.0947, val_loss : 3.3344, acc : 0.667, val_acc : 0.667
Epoch 5, loss : 1.2243, val_loss : 3.3966, acc : 0.667, val_acc : 0.667
Epoch 6, loss : 0.9905, val_loss : 2.9457, acc : 0.667, val_acc : 0.667
Epoch 7, loss : 0.6546, val_loss : 2.5096, acc : 0.667, val_acc : 0.667
Epoch 8, loss : 0.7118, val_loss : 2.6596, acc : 0.667, val_acc : 1.000
Epoch 9, loss : 0.3233, val_loss : 2.2167, acc : 0.667, val_acc : 1.000
test_acc: 0.667
Epoch 0, loss : 29.7817, val_loss : 42.7011, acc : 0.667, val_acc : 0.333
Epoch 1, loss : 3.9112, val_loss : 12.5210, acc : 0.333, val_acc : 0.333
Epoch 2, loss : 15.9963, val_loss : 26.5143, acc : 0.667, val_acc : 0.333
Epoch 3, loss : 15.9284, val_loss : 25.6

## 【問題4】House Pricesのモデルを作成

In [7]:
df = pd.read_csv('../week3/train.csv')
scaler = StandardScaler()
X = scaler.fit_transform(df[['GrLivArea', 'YearBuilt']].values).astype(np.float32)
y = scaler.fit_transform(df[['SalePrice']].values).astype(np.float32)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
from keras import backend as K

Using TensorFlow backend.


In [10]:
num_epochs = 20
learning_rate = 0.0000001
batch_size = 10
n_hidden1 = 100
n_hidden2 = 100
n_input = X_train.shape[1]

X = tf.placeholder('float', [None, n_input])
Y = tf.placeholder('float', [None, 1])

output = example_net(X)
loss_op = tf.reduce_mean(tf.square(Y - output))
train_op = optimizer.minimize(loss_op)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

kf = KFold(n_splits=4, shuffle=True)
for train_idx, valid_idx in kf.split(X_train, y_train):
    X_tr, X_val = X_train[train_idx], X_train[valid_idx]
    y_tr, y_val = y_train[train_idx], y_train[valid_idx]
    n_samples, n_input = X_tr.shape
    get_mini_batch_train = GetMiniBatch(X_tr, y_tr, batch_size=batch_size)
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(num_epochs):
            total_batch  = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
            total_loss = 0
            total_acc = 0
            for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
                sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
                loss = sess.run(loss_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
                total_loss += loss
                
            total_loss /= n_samples
            
            if epoch % 5 == 0:
                val_loss = sess.run(loss_op, feed_dict={X: X_val, Y: y_val})
                print(f"Epoch {epoch}, loss : {loss:.4f}, val_loss : {val_loss:.4f}")

        test_loss = sess.run(loss_op, feed_dict={X: X_test, Y: y_test})
        print(f'test_loss: {test_loss:.3f}')


Epoch 0, loss : 11.2633, val_loss : 11.6100
Epoch 5, loss : 2.5503, val_loss : 5.6638
Epoch 10, loss : 1.6933, val_loss : 3.6318
Epoch 15, loss : 1.3389, val_loss : 4.4258
test_loss: 3.418
Epoch 0, loss : 12.9025, val_loss : 22.6607
Epoch 5, loss : 39.7667, val_loss : 43.1913
Epoch 10, loss : 10.4154, val_loss : 15.9734
Epoch 15, loss : 34.5232, val_loss : 44.5793
test_loss: 6.959
Epoch 0, loss : 9.1323, val_loss : 28.0984
Epoch 5, loss : 4.2526, val_loss : 8.9573
Epoch 10, loss : 3.8737, val_loss : 15.0859
Epoch 15, loss : 2.6550, val_loss : 10.6401
test_loss: 11.831
Epoch 0, loss : 15.6505, val_loss : 26.6835
Epoch 5, loss : 5.3612, val_loss : 14.9287
Epoch 10, loss : 4.8944, val_loss : 9.8719
Epoch 15, loss : 6.5265, val_loss : 19.0731
test_loss: 6.343


## 【問題5】MNISTのモデルを作成

In [11]:
from keras.datasets import mnist
from tqdm import tqdm

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(-1, 28, 28, 1)[:5000]
y_train = y_train[:5000]
X_train = X_train.astype(np.float32)
X_train /= 255

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train_one_hot, test_size=0.2)

In [12]:
def net(x):
    weights = {
        'w1': tf.Variable(tf.random_normal([256, 120])),
        'w2': tf.Variable(tf.random_normal([120, 84])),
        'w3': tf.Variable(tf.random_normal([84, 10])),
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([120])),
        'b2': tf.Variable(tf.random_normal([84])),
        'b3': tf.Variable(tf.random_normal([10])),
    }
    c1 = tf.get_variable('chan1', [5, 5, 1, 6], dtype=tf.float32)
    c2 = tf.get_variable('chan2', [5, 5, 6, 16], dtype=tf.float32)
    
    layer_1 = tf.nn.conv2d(x, filters=c1, strides=1, padding='VALID', data_format='NHWC')
    layer_1 = tf.nn.relu(layer_1)
    layer_1 = tf.nn.max_pool2d(layer_1, ksize=2, strides=2, padding='VALID', data_format='NHWC')
    layer_2 = tf.nn.conv2d(layer_1, filters=c2, strides=1, padding='VALID', data_format='NHWC')
    layer_2 = tf.nn.relu(layer_2)
    layer_2 = tf.nn.max_pool2d(layer_2, ksize=2, strides=2, padding='VALID', data_format='NHWC')
    layer_3 = tf.reshape(layer_2, [-1, 256])
    layer_4 = tf.add(tf.matmul(layer_3, weights['w1']), biases['b1'])
    layer_4 = tf.nn.relu(layer_4)
    layer_5 = tf.add(tf.matmul(layer_4, weights['w2']), biases['b2'])
    layer_5 = tf.nn.relu(layer_5)    
    layer_output = tf.add(tf.matmul(layer_5, weights['w3']), biases['b3'])
    return layer_output

In [13]:
num_epochs = 30
learning_rate = 0.001
batch = 5

X = tf.placeholder('float', [None, 28, 28, 1])
Y = tf.placeholder('float', [None, 10])

logits = net(X)
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
correct_pred = tf.equal(tf.argmax(Y), tf.argmax(tf.nn.softmax(logits)))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

kf = KFold(n_splits=4, shuffle=True)
for train_idx, valid_idx in kf.split(X_train, y_train):
    X_tr, X_val = X_train[train_idx], X_train[valid_idx]
    y_tr, y_val = y_train[train_idx], y_train[valid_idx]
    n_samples = X_tr.shape[0]
    get_mini_batch_train = GetMiniBatch(X_tr, y_tr, batch_size=batch_size)
    
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(num_epochs):
            total_batch  = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
            total_loss = 0
            total_acc = 0
            for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
                sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
                loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
                total_loss += loss
                total_acc += acc
            total_loss /= n_samples
            total_acc /= n_samples
            
            if epoch%5 == 0:
                val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
                print(f"Epoch {epoch}, loss : {loss:.4f}, val_loss : {val_loss:.4f}, acc : {acc:.3f}, val_acc : {val_acc:.3f}")
        test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
        print(f'test_acc: {test_acc:.3f}')

Epoch 0, loss : 0.8832, val_loss : 2.8641, acc : 0.500, val_acc : 0.000
Epoch 5, loss : 0.0082, val_loss : 1.0406, acc : 0.300, val_acc : 0.300
Epoch 10, loss : 0.0107, val_loss : 0.8099, acc : 0.500, val_acc : 0.500
Epoch 15, loss : 0.0000, val_loss : 0.7965, acc : 0.400, val_acc : 0.900
Epoch 20, loss : 0.0000, val_loss : 1.1591, acc : 0.600, val_acc : 0.800
Epoch 25, loss : 0.0000, val_loss : 0.8695, acc : 0.600, val_acc : 1.000
test_acc: 1.000
Epoch 0, loss : 9.3236, val_loss : 5.2649, acc : 0.300, val_acc : 0.300
Epoch 5, loss : 3.0409, val_loss : 2.1967, acc : 0.700, val_acc : 0.500
Epoch 10, loss : 0.4478, val_loss : 1.2594, acc : 0.600, val_acc : 0.600
Epoch 15, loss : 0.0002, val_loss : 1.5522, acc : 0.700, val_acc : 0.600
Epoch 20, loss : 0.0015, val_loss : 0.9612, acc : 0.700, val_acc : 0.700
Epoch 25, loss : 0.0000, val_loss : 1.0686, acc : 0.700, val_acc : 0.700
test_acc: 0.800
Epoch 0, loss : 4.7750, val_loss : 4.7630, acc : 0.500, val_acc : 0.300
Epoch 5, loss : 0.0398, 